In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from constants import BASE_SAVE_DIR, SAVE_EXTENSION, PROJECT_NAME_BASE, FONTSIZE, METRIC_NAME_BASE
from utils import render_in_latex, set_fontsize, get_save_path, get_runs, get_metrics_and_colors, get_metric_statistics, plot_metric_statistics, bar_metric_statistics

In [8]:
datasets = ["houseelec", "malonaldehyde"]
metric = "test_posterior_samples_mean_nll"
full_metric_name = METRIC_NAME_BASE + metric
x_axis_name = "time"

In [9]:
set_fontsize(FONTSIZE)
render_in_latex()

In [10]:
statistics_dicts = {}
for dataset in datasets:
    runs = get_runs(PROJECT_NAME_BASE + dataset)
    metrics_dict, colors_dict = get_metrics_and_colors(runs, metric, full_metric_name)
    statistics_dicts[dataset] = get_metric_statistics(metrics_dict)

save_path_plot = get_save_path(f"{BASE_SAVE_DIR}/all/{metric}", f"{x_axis_name}.{SAVE_EXTENSION}")
save_path_bar = get_save_path(
    f"{BASE_SAVE_DIR}/all/{metric}", f"bar.{SAVE_EXTENSION}")

/home/pratikr/scalable_gp_inference/plotting/utils.py:128: UserWarning: Not all MetricData objects have the same steps. This may lead to incorrect results. This is likely because some runs were not finished. We will return None for the mean, min, and max data.
  warnings.warn(
/home/pratikr/scalable_gp_inference/plotting/utils.py:202: UserWarning: Skipping SDD-100 due to inconsistent metric data across runs.
  warnings.warn(


In [11]:
plot_metric_statistics(statistics_dicts, colors_dict, x_axis_name, save_path=save_path_plot)

In [12]:
bar_metric_statistics(statistics_dicts, colors_dict, save_path=save_path_bar)